In [1]:
%pip install ipywidgets
%pip install tensorflow_datasets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf

def mnist_to_convolutional(image, label):
    image = tf.pad(image, [[2, 2], [2, 2], [0, 0]])
    image = tf.cast(image, tf.float32) / 255.0
    return image, image

train_shape = (32,32,1)

(ds_train, ds_test), ds_info = tfds.load(
    'smallnorb',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# ds_train = ds_train.map(
#     mnist_to_convolutional,
#     num_parallel_calls=tf.data.AUTOTUNE
# )
# ds_train = ds_train.cache()
# ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
# ds_train = ds_train.batch(128)
# ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

# ds_test = ds_test.map(
#     mnist_to_convolutional,
#     num_parallel_calls=tf.data.AUTOTUNE
# )
# ds_test = ds_test.batch(128)
# ds_test = ds_test.cache()
# ds_test = ds_test.prefetch(tf.data.AUTOTUNE)


In [3]:
import tensorflow as tf

encoder = tf.keras.Sequential([
    tf.keras.layers.Input(shape=train_shape),
    tf.keras.layers.Conv2D(
        filters=16,
        kernel_size=3,
        activation=tf.keras.activations.relu,
        padding="same",
        strides=2,
    ),
    tf.keras.layers.Conv2D(
        filters=8,
        kernel_size=3,
        activation=tf.keras.activations.relu,
        padding="same",
        strides=2,
    ),
])

decoder = tf.keras.Sequential([
    tf.keras.layers.Input(shape=encoder.output_shape[1:]),
    tf.keras.layers.Conv2DTranspose(
        filters=8,
        kernel_size=3,
        activation=tf.keras.activations.relu,
        padding="same",
        strides=2,
    ),
    tf.keras.layers.Conv2DTranspose(
        filters=16,
        kernel_size=3,
        activation=tf.keras.activations.relu,
        padding="same",
        strides=2,
    ),
    tf.keras.layers.Conv2D(
        filters=1,
        kernel_size=3,
        activation=tf.keras.activations.sigmoid,
        padding="same",
        strides=1
    ),
])

inputs = tf.keras.layers.Input(shape=train_shape)

outputs = decoder(encoder(inputs))

autoencoder = tf.keras.Model(inputs=inputs, outputs=outputs)

autoencoder.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=tf.keras.optimizers.Adam(),
)

print(train_shape, tf.math.reduce_prod(train_shape))
print(encoder.output_shape[1:], tf.math.reduce_prod(encoder.output_shape[1:]))
print(decoder.output_shape[1:], tf.math.reduce_prod(decoder.output_shape[1:]))


(32, 32, 1) tf.Tensor(1024, shape=(), dtype=int32)
(8, 8, 8) tf.Tensor(512, shape=(), dtype=int32)
(32, 32, 1) tf.Tensor(1024, shape=(), dtype=int32)


In [4]:
last_loss = float('inf')
last_val_loss = float('inf')
threshold = 0.8  # 20% improvement threshold
patience = 10

while (patience > 0):
    patience -= 1
    history = autoencoder.fit(
        ds_train,
        epochs=1,
        batch_size=32,
        shuffle=True,
        validation_data=ds_test,
    )
    loss = history.history['loss'][-1]
    val_loss = history.history['val_loss'][-1]
    if loss < last_loss * threshold or val_loss < last_val_loss * threshold:
        last_loss = loss
        last_val_loss = val_loss
    else:
        break

ValueError: in user code:

    File "c:\Users\Frederik\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Frederik\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Frederik\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Frederik\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Frederik\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

test_samples = np.array([
    image
    for batch, _ in ds_test.as_numpy_iterator()
    for image in batch
])

encoded_samples = encoder.predict(test_samples)
decoded_samples = decoder.predict(encoded_samples)

n = 100
plt.figure(figsize=(n * 2, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(test_samples[i])
    plt.title("original")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_samples[i])
    plt.title("decoded")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()


In [ ]:
plt.scatter([point[0] for point in encoded_samples], [point[1] for point in encoded_samples])
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.show()


In [ ]:
import ipywidgets

#@ipywidgets.interact(x=(np.min(encoded_samples[:, 0]), np.max(encoded_samples[:, 0])), y=(np.min(encoded_samples[:, 1]), np.max(encoded_samples[:, 1])))
def generate_image(x=0.0, y=0.0):
    plt.imshow(decoder(np.array([[x, y]]))[0])
    plt.show()

generate_image(x=20.0, y=-20.0)